# ICU Length of Stay Prediction - MIMIC-III Pipeline

## 🎯 Objective
Predict ICU stay duration using PySpark ML on MIMIC-III dataset

## 📊 Data & Constraints
- **Sources**: 6 MIMIC-III tables (CHARTEVENTS, LABEVENTS, ICUSTAYS, etc.)
- **Filters**: 
        - Patient Age 18-80
        - LOS 0.1-15 days
        - Valid time sequences
- **Timeframe**: Vitals (first 24h), Labs (6h pre to 24h post ICU)


## 🌀 Big Data Processing

- **CHARTEVENTS**: Chart Events table has +330 million rows
- **Parquet**: Converted to Parquet format for efficient storage and processing
- **Filtering**: We filtered immediately when loading to optimize CHARTEVENTS DataFrame

## 🔧 Features (39 total)
- **Demographics (2)**: Age, gender
- **Admission (8)**: Emergency/elective, timing, insurance
- **ICU Units (6)**: Care unit types, transfers
- **Vitals (11)**: HR, BP, RR, temp, SpO2 (avg/std)
- **Labs (8)**: Creatinine, glucose, electrolytes, blood counts
- **Diagnoses (4)**: Total count, sepsis, respiratory failure

## 🤖 Models & Results
- **Linear Regression**: 
- **Random Forest**: 

## ☁️ Infrastructure
- **GCP Dataproc**: 6x e2-highmem-4 workers (28 vCPUs, 224GB RAM)
- **Optimizations**: Smart sampling, aggressive filtering, 80/20 split





## Import Libraries

In [148]:
# Core PySpark imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

# Machine Learning imports
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer
from pyspark.ml.regression import RandomForestRegressor, LinearRegression #, GBTRegressor
#from pyspark.ml.classification import RandomForestClassifier, LogisticRegression
from pyspark.ml.evaluation import RegressionEvaluator #, MulticlassClassificationEvaluator
#from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
#from pyspark.ml import Pipeline

from datetime import datetime, timedelta
import time

print("✅ All imports loaded successfully!")
print(f"⏰ Notebook started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ All imports loaded successfully!
⏰ Notebook started at: 2025-06-02 13:59:52


## Setup Spark Session

In [149]:
# spark = SparkSession.builder \
#     .appName("Forecast-LOS") \
#     .config("spark.sql.adaptive.enabled", "true") \
#     .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
#     .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
#     .config("spark.sql.shuffle.partitions", "200") \
#     .config("spark.sql.adaptive.skewJoin.enabled", "true") \
#     .config("spark.sql.adaptive.localShuffleReader.enabled", "true") \
#     .config("spark.sql.adaptive.advisoryPartitionSizeInBytes", "128MB") \
#     .config("spark.sql.adaptive.coalescePartitions.minPartitionSize", "64MB") \
#     .config("spark.sql.adaptive.skewJoin.skewedPartitionThresholdInBytes", "128MB") \
#     .config("spark.sql.files.maxPartitionBytes", "64MB") \
#     .config("spark.network.timeout", "1200s") \
#     .config("spark.executor.heartbeatInterval", "120s") \
#     .config("spark.executor.memory", "12g") \
#     .config("spark.executor.cores", "2") \
#     .config("spark.executor.instances", "8") \
#     .config("spark.driver.memory", "4g") \
#     .config("spark.driver.maxResultSize", "2g") \
#     .config("spark.sql.execution.arrow.maxRecordsPerBatch", "1000") \
#     .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
#     .config("spark.sql.adaptive.skewJoin.skewedPartitionFactor", "5") \
#     .config("spark.sql.adaptive.nonEmptyPartitionRatioForBroadcastJoin", "0.2") \
#     .config("spark.dynamicAllocation.enabled", "false") \
#     .config("spark.sql.broadcastTimeout", "36000") \
#     .config("spark.rpc.askTimeout", "600s") \
#     .config("spark.network.timeoutInterval", "120s") \
#     .config("spark.storage.blockManagerSlaveTimeoutMs", "120000") \
#     .config("spark.executor.heartbeatInterval", "20s") \
#     .config("spark.network.timeout", "120s") \
#     .config("spark.rpc.lookupTimeout", "120s") \
#     .config("spark.shuffle.registration.timeout", "120000") \
#     .config("spark.shuffle.registration.maxAttempts", "5") \
#     .getOrCreate()

spark = SparkSession.builder \
    .appName("Forecast-LOS") \
    .getOrCreate()

print("✅ Spark session created successfully!")
print(f"📊 Spark Version: {spark.version}")
print(f"🔧 Application Name: {spark.sparkContext.appName}")
print(f"💾 Available cores: {spark.sparkContext.defaultParallelism}")
print(f"\n⏰ Spark session initialised at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Spark session created successfully!
📊 Spark Version: 3.5.5
🔧 Application Name: Forecast-LOS
💾 Available cores: 8

⏰ Spark session initialised at: 2025-06-02 13:59:52


# Load Data

In [150]:
# Configuration flags
SAMPLE_ENABLE = True
SAMPLE_SIZE = 100

#MIMIC_PATH = "gs://dataproc-staging-europe-west2-851143487985-hir6gfre/mimic-data"
#MIMIC_PATH = "./mimic-data"
MIMIC_PATH =  "./mimic-db-short"

print("🏥 Loading MIMIC-III CSV files...")

# Step 1: Load and sample ICUSTAYS first
print("📂 Loading ICUSTAYS table...")
icustays_df = spark.read.option("header", "true").option("inferSchema", "true").csv(f"{MIMIC_PATH}/ICUSTAYS.csv")

if SAMPLE_ENABLE:
    print(f"🎯 Sampling {SAMPLE_SIZE} ICU stays...")
    icustays_df = icustays_df.limit(SAMPLE_SIZE) 
    icustays_df.cache()
    actual_sample_size = icustays_df.count()
    print(f"✅ Sampled {actual_sample_size} ICU stays")

# Step 2: Get required IDs for filtering other tables
print("📋 Extracting required IDs...")
icu_ids = icustays_df.select("ICUSTAY_ID").rdd.map(lambda row: row[0]).collect()
hadm_ids = icustays_df.select("HADM_ID").rdd.map(lambda row: row[0]).collect()
subject_ids = icustays_df.select("SUBJECT_ID").rdd.map(lambda row: row[0]).collect()

print(f"📊 IDs to filter: {len(icu_ids)} ICUSTAY_IDs, {len(hadm_ids)} HADM_IDs, {len(subject_ids)} SUBJECT_IDs")

# Step 3: Load other tables with pre-filtering
print("📂 Loading PATIENTS table...")
patients_df = spark.read.option("header", "true").option("inferSchema", "true").csv(f"{MIMIC_PATH}/PATIENTS.csv")
patients_df = patients_df.filter(col("SUBJECT_ID").isin(subject_ids))

print("📂 Loading ADMISSIONS table...")
admissions_df = spark.read.option("header", "true").option("inferSchema", "true").csv(f"{MIMIC_PATH}/ADMISSIONS.csv")
admissions_df = admissions_df.filter(col("HADM_ID").isin(hadm_ids))

print("📂 Loading DIAGNOSES_ICD table...")
diagnoses_df = spark.read.option("header", "true").option("inferSchema", "true").csv(f"{MIMIC_PATH}/DIAGNOSES_ICD.csv")
diagnoses_df = diagnoses_df.filter(col("HADM_ID").isin(hadm_ids))




# Step 4: Load large tables (CHARTEVENTS, LABEVENTS) with aggressive filtering
print("📂 Loading CHARTEVENTS table... [FILTERING BY ICUSTAY_ID]")

try:
    chartevents_df = spark.read.parquet(f"{MIMIC_PATH}/CHARTEVENTS.parquet")
except:
    print(f" Converting CHARTEVENTS.csv.gz to parquet...")
    chartevents_csv = spark.read.option("header", "true").option("inferSchema", "false").csv(f"{MIMIC_PATH}/CHARTEVENTS.csv")
    chartevents_csv.write.mode("overwrite").parquet(f"{MIMIC_PATH}/CHARTEVENTS.parquet")
    chartevents_df = spark.read.parquet(f"{MIMIC_PATH}/CHARTEVENTS.parquet")
    
icu_ids_df = spark.createDataFrame([(id,) for id in icu_ids], ["ICUSTAY_ID"])
chartevents_df = chartevents_df \
    .select("ICUSTAY_ID", "CHARTTIME", "ITEMID", "VALUE", "VALUEUOM", "VALUENUM") \
    .join(broadcast(icu_ids_df), "ICUSTAY_ID", "inner")




print("📂 Loading LABEVENTS table... [FILTERING BY HADM_ID]")

try:
    labevents_df = spark.read.parquet(f"{MIMIC_PATH}/LABEVENTS.parquet")
except:
    print(f" Converting LABEVENTS.csv.gz to parquet...")
    labevents_csv = spark.read.option("header", "true").option("inferSchema", "false").csv(f"{MIMIC_PATH}/LABEVENTS.csv")
    labevents_csv.write.mode("overwrite").parquet(f"{MIMIC_PATH}/LABEVENTS.parquet")
    labevents_df = spark.read.parquet(f"{MIMIC_PATH}/LABEVENTS.parquet")

    
labevents_df = labevents_df \
                .filter(col("HADM_ID").isin(hadm_ids))




# Display final information
print("\n✅ Tables loaded and filtered successfully!")
if SAMPLE_ENABLE:
    print(f"🎯 Using sample of {actual_sample_size} ICU stays")
    print(f"📊 ICUSTAYS: {icustays_df.count():,} rows")
    print(f"📊 PATIENTS: {patients_df.count():,} rows") 
    print(f"📊 ADMISSIONS: {admissions_df.count():,} rows")
    print(f"📊 DIAGNOSES_ICD: {diagnoses_df.count():,} rows")
    print(f"📊 CHARTEVENTS (filtered): {chartevents_df.count():,} rows")
    print(f"📊 LABEVENTS (filtered): {labevents_df.count():,} rows")

print(f"\n⏰ Data loaded at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

🏥 Loading MIMIC-III CSV files...
📂 Loading ICUSTAYS table...


🎯 Sampling 100 ICU stays...
✅ Sampled 20 ICU stays
📋 Extracting required IDs...


25/06/02 13:59:52 WARN CacheManager: Asked to cache already cached data.


📊 IDs to filter: 20 ICUSTAY_IDs, 20 HADM_IDs, 20 SUBJECT_IDs
📂 Loading PATIENTS table...
📂 Loading ADMISSIONS table...
📂 Loading DIAGNOSES_ICD table...
📂 Loading CHARTEVENTS table... [FILTERING BY ICUSTAY_ID]
📂 Loading LABEVENTS table... [FILTERING BY HADM_ID]

✅ Tables loaded and filtered successfully!
🎯 Using sample of 20 ICU stays
📊 ICUSTAYS: 20 rows
📊 PATIENTS: 20 rows
📊 ADMISSIONS: 20 rows
📊 DIAGNOSES_ICD: 212 rows


📊 CHARTEVENTS (filtered): 57,973 rows
📊 LABEVENTS (filtered): 5,895 rows

⏰ Data loaded at: 2025-06-02 13:59:58


## Features Engineering

Current features for regression:

- Demographics (age, gender)
- Admission characteristics (emergency vs elective, timing)
- ICU unit types and transfers
- Time-based features (weekend, night admissions)
- Medical data


## Extracting Data From ICUSTAYS

In [151]:
print("📊 Step 1: Creating base ICU dataset with patient demographics...")

base_icu_df = icustays_df.alias("icu") \
    .join(patients_df.alias("pat"), "SUBJECT_ID", "inner") \
    .join(admissions_df.alias("adm"), ["SUBJECT_ID", "HADM_ID"], "inner") \
    .select(
        # ICU stay identifiers
        col("icu.ICUSTAY_ID"),
        col("icu.SUBJECT_ID"), 
        col("icu.HADM_ID"),
        
        # Target variable - Length of Stay in ICU (days)
        col("icu.LOS").alias("ICU_LOS"),
        
        # ICU characteristics
        col("icu.FIRST_CAREUNIT"),
        col("icu.LAST_CAREUNIT"), 
        col("icu.INTIME").alias("ICU_INTIME"),
        col("icu.OUTTIME").alias("ICU_OUTTIME"),
        
        # Patient demographics
        col("pat.GENDER"),
        col("pat.DOB"),
        col("pat.EXPIRE_FLAG").alias("PATIENT_DIED"),
        
        # Admission details
        col("adm.ADMITTIME"),
        col("adm.DISCHTIME"), 
        col("adm.ADMISSION_TYPE"),
        col("adm.ADMISSION_LOCATION"),
        col("adm.INSURANCE"),
        col("adm.ETHNICITY"),
        col("adm.HOSPITAL_EXPIRE_FLAG").alias("HOSPITAL_DEATH"),
        col("adm.DIAGNOSIS").alias("ADMISSION_DIAGNOSIS")
    )

# Calculate age at ICU admission
base_icu_df = base_icu_df.withColumn("AGE_AT_ICU_ADMISSION", \
                                     floor(datediff(col("ICU_INTIME"), col("DOB")) / 365.25)) \
                                     .filter(col("AGE_AT_ICU_ADMISSION").between(18,80))


print("✅ Created base ICU dataset!")


📊 Step 1: Creating base ICU dataset with patient demographics...
✅ Created base ICU dataset!


## Extracting Categorical Features

In [152]:
print("📊 Step 2: Engineering categorical features...")
base_icu_df = base_icu_df \
    .withColumn("GENDER_BINARY", when(col("GENDER") == "M", 1).otherwise(0)) \
    .withColumn("CAME_FROM_ER", when(col("ADMISSION_LOCATION").contains("EMERGENCY"), 1).otherwise(0)) \
    .withColumn("HAS_INSURANCE", when(col("INSURANCE") == "Medicare", 1).otherwise(0)) \
    .withColumn("ADMISSION_TYPE_ENCODED", 
                when(col("ADMISSION_TYPE") == "EMERGENCY", 1)
                .when(col("ADMISSION_TYPE") == "ELECTIVE", 2)
                .when(col("ADMISSION_TYPE") == "URGENT", 3)
                .otherwise(0)) \
    .withColumn("ETHNICITY_ENCODED",
                when(col("ETHNICITY").contains("WHITE"), 1) \
                .when(col("ETHNICITY").contains("BLACK"), 2) \
                .when(col("ETHNICITY").contains("HISPANIC"), 3) \
                .when(col("ETHNICITY").contains("ASIAN"), 4) \
                .otherwise(5))

print("✅ Base ICU Dataset - Categorical Features")


📊 Step 2: Engineering categorical features...


✅ Base ICU Dataset - Categorical Features


## Extracting ICU Unit Types

In [153]:
print("📊 Step 3: Creating ICU unit type features...")

base_icu_df = base_icu_df \
    .withColumn("FIRST_UNIT_ENCODED", 
                when(col("FIRST_CAREUNIT") == "MICU", 1)
                .when(col("FIRST_CAREUNIT") == "SICU", 2)
                .when(col("FIRST_CAREUNIT") == "CSRU", 3)
                .when(col("FIRST_CAREUNIT") == "CCU", 4)
                .when(col("FIRST_CAREUNIT") == "TSICU", 5)
                .otherwise(0)) \
    .withColumn("CHANGED_ICU_UNIT", 
                when(col("FIRST_CAREUNIT") != col("LAST_CAREUNIT"), 1).otherwise(0))


print("✅ Base ICU Dataset - Unit Type Features")

📊 Step 3: Creating ICU unit type features...
✅ Base ICU Dataset - Unit Type Features


## Extracting Time-based Features

In [154]:
print("📊 Step 4: Creating time-based features...")
base_icu_df = base_icu_df \
    .filter(col("ICU_INTIME") < col("ICU_OUTTIME"))
print("✅ Base ICU Dataset - Time Based Features")

📊 Step 4: Creating time-based features...
✅ Base ICU Dataset - Time Based Features


## Remove Outliers (Excessive Length Of Stay)

In [155]:
print("\n📈 ICU Length of Stay Statistics (Days):")
base_icu_df.select("ICU_LOS").describe().show()


📈 ICU Length of Stay Statistics (Days):
+-------+-----------------+
|summary|          ICU_LOS|
+-------+-----------------+
|  count|               16|
|   mean|        2.2869125|
| stddev|2.035743709106494|
|    min|            0.758|
|    max|           8.9163|
+-------+-----------------+



We kept every icu stay that had duration between 0.0 and 9.1 days, considered normal legnths since:

| Statistic                | Value (days)                                    |
| ------------------------ | ----------------------------------------------- |
| **Minimum**              | 0.0 (can be admission + discharge on same day)  |
| **25th percentile (Q1)** | \~1.1                                           |
| **Median (Q2)**          | \~2.1                                           |
| **75th percentile (Q3)** | \~4.3                                           |
| **Maximum**              | \~88 (but can go slightly higher in edge cases) |
| **Mean**                 | \~3.3–3.5                                       |

Using interquartile range (IQR) method:

* IQR = Q3 - Q1 = 4.3 - 1.1 = ~3.2

* Upper Bound for outliers = Q3 + 1.5 × IQR ≈ 4.3 + 4.8 = ~9.1 days

* Lower Bound = Q1 - 1.5 × IQR ≈ 1.1 - 4.8 = < 0, which is ignored since LOS can’t be negative

So:

* Typical ICU LOS: 1.1 to 4.3 days

* Outliers: ICU stays longer than ~9.1 days


In [156]:
print("📊 Step 5: Cleaning target variable...")

# List of desired columns
selected_columns = [
    "ICUSTAY_ID", "SUBJECT_ID", "HADM_ID", "ICU_LOS", "ICU_INTIME", "ICU_OUTTIME", "AGE_AT_ICU_ADMISSION", "GENDER_BINARY", "CAME_FROM_ER",
    "HAS_INSURANCE", "ADMISSION_TYPE_ENCODED", "ETHNICITY_ENCODED",
    "FIRST_UNIT_ENCODED", "CHANGED_ICU_UNIT"
]

# Apply filter and select columns
base_icu_df = base_icu_df \
    .filter(col("ICU_LOS").between(0.0, 9.1)) \
    .select(*selected_columns) \
    .cache()

print("✅ Base ICU Dataset - Remove Outliers")

print("\n📋 Sample of ICU stay records:")
base_icu_df.show(5)

📊 Step 5: Cleaning target variable...
✅ Base ICU Dataset - Remove Outliers

📋 Sample of ICU stay records:
+----------+----------+-------+-------+-------------------+-------------------+--------------------+-------------+------------+-------------+----------------------+-----------------+------------------+----------------+
|ICUSTAY_ID|SUBJECT_ID|HADM_ID|ICU_LOS|         ICU_INTIME|        ICU_OUTTIME|AGE_AT_ICU_ADMISSION|GENDER_BINARY|CAME_FROM_ER|HAS_INSURANCE|ADMISSION_TYPE_ENCODED|ETHNICITY_ENCODED|FIRST_UNIT_ENCODED|CHANGED_ICU_UNIT|
+----------+----------+-------+-------+-------------------+-------------------+--------------------+-------------+------------+-------------+----------------------+-----------------+------------------+----------------+
|    255819|      1452| 156406|  0.758|2129-05-10 21:36:40|2129-05-11 15:48:11|                  56|            0|           0|            0|                     1|                1|                 1|               1|
|    231977|      

## Extracting Clinical Features

Get top 20 more common chart events, usually vital signs, calculate the avg of each test in the first 24h in the ICU. If a person did not do that test then the resulting value should be read -1 and not null, this ensures compatibility with ML algorithms that don’t handle missing values well.

In [157]:
print("📊 Identifying top 20 most frequent tests from CHARTEVENTS...")

# Get frequency count of each ITEMID in CHARTEVENTS
itemid_counts = chartevents_df \
    .filter(col("ICUSTAY_ID").isin(icu_ids_list)) \
    .filter(col("VALUENUM").isNotNull()) \
    .filter(col("CHARTTIME").isNotNull()) \
    .groupBy("ITEMID") \
    .count() \
    .orderBy(col("count").desc()) \
    .limit(20) \
    .collect()

# Create mapping dictionary for top 20 items
top_20_items = {row["ITEMID"]: f"VITAL_{row['ITEMID']}" for row in itemid_counts}
print(f"🎯 Top 20 chart items selected: {top_20_items}")

print("📊 Filtering CHARTEVENTS for top 20 items...")

chartevents_top20 = chartevents_df \
    .filter(col("ITEMID").isin(list(top_20_items.keys()))) \
    .filter(col("VALUENUM").isNotNull()) \
    .filter(col("VALUENUM").isNotNull()) \
    .filter(col("ICUSTAY_ID").isin(icu_ids_list)) \
    .filter(col("CHARTTIME").isNotNull()) \
    .join(base_icu_df.select("ICUSTAY_ID", "ICU_INTIME", "ICU_OUTTIME"), "ICUSTAY_ID", "inner") \
    .filter(col("CHARTTIME").between(col("ICU_INTIME"), col("ICU_OUTTIME"))) \
    .select("ICUSTAY_ID", "ITEMID", "CHARTTIME", "VALUENUM")

# Process first 24 hours
vitals_24h_top20 = chartevents_top20.alias("ce") \
    .join(base_icu_df.select("ICUSTAY_ID", "ICU_INTIME"), "ICUSTAY_ID", "inner") \
    .filter(
        col("ce.CHARTTIME").between(
            col("ICU_INTIME"), 
            col("ICU_INTIME") + expr("INTERVAL 24 HOURS")
        )
    )

print("📊 Calculating aggregates for top 20 vitals...")

# Initialize with ICUSTAY_ID
vitals_features_top20 = base_icu_df.select("ICUSTAY_ID")

# Process each vital sign
for itemid, name in top_20_items.items():
    #print(f"Processing {name} (ITEMID={itemid})...")
    
    vital_stats = vitals_24h_top20 \
        .filter(col("ITEMID") == itemid) \
        .groupBy("ICUSTAY_ID") \
        .agg(avg("VALUENUM").alias(f"{name}_AVG"))
    
    # Left join (without filling NULLs yet)
    vitals_features_top20 = vitals_features_top20.join(vital_stats, "ICUSTAY_ID", "left")

# Fill ALL NULL values with -1 AFTER all joins are done
vitals_features_top20 = vitals_features_top20.na.fill(-1)

# Cleanup
chartevents_df.unpersist()
vitals_24h_top20.unpersist()

# Verify no NULLs remain
print(f"✅ Created {len(top_20_items)} features from top 20 vital signs (NULLs replaced with -1)")
vitals_features_top20.show(5)

📊 Identifying top 20 most frequent tests from CHARTEVENTS...
🎯 Top 20 chart items selected: {'220045': 'VITAL_220045', '220277': 'VITAL_220277', '220210': 'VITAL_220210', '220181': 'VITAL_220181', '220179': 'VITAL_220179', '220180': 'VITAL_220180', '223901': 'VITAL_223901', '220739': 'VITAL_220739', '223900': 'VITAL_223900', '220050': 'VITAL_220050', '220052': 'VITAL_220052', '220051': 'VITAL_220051', '223753': 'VITAL_223753', '220074': 'VITAL_220074', '224168': 'VITAL_224168', '224641': 'VITAL_224641', '223761': 'VITAL_223761', '223791': 'VITAL_223791', '220765': 'VITAL_220765', '211': 'VITAL_211'}
📊 Filtering CHARTEVENTS for top 20 items...
📊 Calculating aggregates for top 20 vitals...
✅ Created 20 features from top 20 vital signs (NULLs replaced with -1)
+----------+------------------+----------------+------------------+-----------------+------------------+-----------------+----------------+----------------+----------------+------------------+-----------------+-----------------+----

Get top 20 more common lab events, calculate the avg of each test in the first 24h in the ICU and the 6h prior to it. If a person did not do that test then the resulting value should be read -1 and not null.

In [158]:
print("\n🧪 Creating laboratory features from LABEVENTS...")

# Step 1: Identify top 20 most frequent lab items
print("📊 Identifying top 20 most frequent lab items...")
top_20_lab_items = labevents_df \
    .filter(col("HADM_ID").isin([row["HADM_ID"] for row in base_icu_df.select("HADM_ID").collect()])) \
    .filter(col("VALUENUM").isNotNull()) \
    .filter(col("VALUENUM") > 0) \
    .groupBy("ITEMID") \
    .count() \
    .orderBy(col("count").desc()) \
    .limit(20) \
    .collect()

# Create mapping dictionary (using ITEMID as name if no mapping exists)
lab_items = {row["ITEMID"]: f"LAB_{row['ITEMID']}" for row in top_20_lab_items}
print(f"🎯 Top 20 lab items selected: {list(lab_items.keys())}")

# Step 2: Filter lab events within first 24 hours of ICU stay
print("📊 Filtering LABEVENTS for top 20 items...")
labs_24h = labevents_df.alias("le") \
    .join(base_icu_df.select("ICUSTAY_ID", "HADM_ID", "ICU_INTIME"), "HADM_ID", "inner") \
    .filter(col("le.ITEMID").isin(list(lab_items.keys()))) \
    .filter(col("le.VALUENUM").isNotNull()) \
    .filter(col("le.VALUENUM") > 0) \
    .filter(
        col("le.CHARTTIME").between(
            col("ICU_INTIME") - expr("INTERVAL 6 HOURS"),  # Include pre-ICU labs
            col("ICU_INTIME") + expr("INTERVAL 24 HOURS")
        )
    )

# Step 3: Calculate lab statistics with NULL handling
print("📊 Calculating laboratory statistics...")
labs_features = base_icu_df.select("ICUSTAY_ID")

for itemid, name in lab_items.items():
    #print(f"   Processing {name} (ITEMID={itemid})...")
    
    item_stats = labs_24h \
        .filter(col("ITEMID") == itemid) \
        .groupBy("ICUSTAY_ID") \
        .agg(
            coalesce(avg("VALUENUM"), lit(-1)).alias(f"{name}_AVG")
        )
    
    labs_features = labs_features.join(item_stats, "ICUSTAY_ID", "left")

# Final NULL fill as safeguard (though coalesce should have handled it)
labs_features = labs_features.na.fill(-1)

# Cleanup
labevents_df.unpersist()
labs_24h.unpersist()

print(f"✅ Created {len(lab_items)} lab features for {labs_features.count():,} ICU stays")

# Show sample of features
print("📊 Sample features:")
labs_features.show(5, truncate=False)


🧪 Creating laboratory features from LABEVENTS...
📊 Identifying top 20 most frequent lab items...
🎯 Top 20 lab items selected: ['51221', '50983', '50971', '51265', '51222', '51279', '51249', '51248', '51277', '51250', '51006', '51301', '50902', '50882', '50868', '50931', '50960', '50893', '50970', '51275']
📊 Filtering LABEVENTS for top 20 items...
📊 Calculating laboratory statistics...
✅ Created 20 lab features for 16 ICU stays
📊 Sample features:
+----------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+-------------+------------------+------------------+-------------+-----------------+------------------+-------------+-------------+-----------------+
|ICUSTAY_ID|LAB_51221_AVG     |LAB_50983_AVG     |LAB_50971_AVG     |LAB_51265_AVG     |LAB_51222_AVG    |LAB_51279_AVG     |LAB_51249_AVG     |LAB_51248_AVG     |LAB_51277_AVG    

In [159]:
print("\n🏥 Creating diagnosis features from ICD codes...")

# Step 1: Identify top 10 most frequent ICD9 codes
print("📊 Identifying top 10 most frequent diagnoses...")
top_10_diagnoses = diagnoses_df \
    .groupBy("ICD9_CODE") \
    .count() \
    .orderBy(col("count").desc()) \
    .limit(10) \
    .collect()

top_10_codes = [row["ICD9_CODE"] for row in top_10_diagnoses]
print(f"🎯 Top 10 ICD9 codes: {top_10_codes}")

# Step 2: Count total diagnoses per admission (comorbidity burden)
diagnosis_features = diagnoses_df.groupBy("HADM_ID") \
    .agg(
        count("ICD9_CODE").alias("TOTAL_DIAGNOSES"),
        collect_list("ICD9_CODE").alias("DIAGNOSIS_CODES")
    )

# Step 3: Create binary features for top 10 diagnoses
for code in top_10_codes:
    diagnosis_features = diagnosis_features.withColumn(
        f"HAS_{code}",
        when(array_contains(col("DIAGNOSIS_CODES"), code), 1).otherwise(0)
    )

# Drop the raw codes list
diagnosis_features = diagnosis_features.drop("DIAGNOSIS_CODES")

print(f"✅ Created {len(top_10_codes)} diagnosis features for {diagnosis_features.count():,} admissions")

# Show sample of features
print("📊 Sample features:")
diagnosis_features.show(5, truncate=False)

print(f"\n⏰ Clinical features completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


🏥 Creating diagnosis features from ICD codes...
📊 Identifying top 10 most frequent diagnoses...
🎯 Top 10 ICD9 codes: ['4019', '42731', '2724', '5849', '0389', '2449', '2851', '25000', '486', '99591']
✅ Created 10 diagnosis features for 20 admissions
📊 Sample features:
+-------+---------------+--------+---------+--------+--------+--------+--------+--------+---------+-------+---------+
|HADM_ID|TOTAL_DIAGNOSES|HAS_4019|HAS_42731|HAS_2724|HAS_5849|HAS_0389|HAS_2449|HAS_2851|HAS_25000|HAS_486|HAS_99591|
+-------+---------------+--------+---------+--------+--------+--------+--------+--------+---------+-------+---------+
|178506 |6              |0       |0        |0       |0       |0       |0       |0       |0        |1      |0        |
|117313 |12             |1       |0        |1       |0       |1       |0       |0       |0        |0      |1        |
|177309 |16             |1       |1        |0       |0       |0       |0       |1       |0        |0      |1        |
|187714 |6            

# Joining All Features

In [162]:
print("📊 Joining all features...")

# Start with base ICU dataset and join all features
final_dataset = base_icu_df \
    .join(vitals_features_top20, "ICUSTAY_ID", "left") \
    .join(labs_features, "ICUSTAY_ID", "left") \
    .join(diagnosis_features, "HADM_ID", "left")

# Cleanup
base_icu_df.unpersist()
vitals_features_top20.unpersist()
labs_features.unpersist()
diagnosis_features.unpersist()

print(f"✅ All features joined! Final dataset has {final_dataset.count()} records")
print("📊 Sample of final dataset:")
final_dataset.show(5, truncate=False)

📊 Joining all features...


ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
print("\n📋 Step 3: Selecting final features for regression modeling...")

# Define feature columns for modeling
feature_columns = [
    # Demographics
    "AGE_AT_ICU_ADMISSION", "GENDER_BINARY",
    
    # Admission characteristics
    "IS_EMERGENCY_ADMISSION", "IS_ELECTIVE_ADMISSION", "CAME_FROM_ER",
    "HAS_MEDICARE", "IS_WHITE_ETHNICITY", "ADMISSION_TO_ICU_HOURS",
    "WEEKEND_ADMISSION", "NIGHT_ADMISSION",
    
    # ICU unit features
    "FIRST_UNIT_MICU", "FIRST_UNIT_SICU", "FIRST_UNIT_CSRU", 
    "FIRST_UNIT_CCU", "FIRST_UNIT_TSICU", "CHANGED_ICU_UNIT",
    
    # Vital signs (averages)
    "HEART_RATE_AVG", "SBP_AVG", "DBP_AVG", "RESP_RATE_AVG", 
    "TEMPERATURE_AVG", "SPO2_AVG",
    
    # Vital signs (variability)
    "HEART_RATE_STD", "SBP_STD", "DBP_STD", "RESP_RATE_STD", "SPO2_STD",
    
    # Laboratory values
    "CREATININE_FIRST", "GLUCOSE_FIRST", "SODIUM_FIRST", "POTASSIUM_FIRST",
    "HEMOGLOBIN_FIRST", "PLATELET_FIRST", "WBC_FIRST", "PH_FIRST",
    
    # Diagnosis features
    "TOTAL_DIAGNOSES", "HAS_SEPSIS", "HAS_RESPIRATORY_FAILURE", "HAS_CARDIAC_ARREST"
]

# Create modeling dataset with selected features
modeling_dataset = final_dataset.select(
    ["ICUSTAY_ID", "ICU_LOS_DAYS"] + feature_columns
)

# Remove any remaining nulls and invalid records
modeling_dataset = modeling_dataset.filter(col("ICU_LOS_DAYS").isNotNull()) \
    .filter(col("ICU_LOS_DAYS") > 0) \
    .filter(col("AGE_AT_ICU_ADMISSION").between(18,80))

# Cache the final dataset
#modeling_dataset = modeling_dataset.repartition()
#modeling_dataset.cache()


print(f"✅ Final modeling dataset prepared!")
print(f"📏 Final dataset: {modeling_dataset.count():,} ICU stays")
print(f"📊 Total features: {len(feature_columns)} predictive features")
print(f"🎯 Target variable: ICU_LOS_DAYS (continuous)")

# Show feature summary
print(f"\n📋 Feature categories:")
print(f"   👤 Demographics: 2 features")
print(f"   🏥 Admission: 8 features") 
print(f"   🏢 ICU Unit: 6 features")
print(f"   🫀 Vital Signs: 11 features")
print(f"   🧪 Laboratory: 8 features")
print(f"   🩺 Diagnoses: 4 features")

# Display sample of final dataset
#print(f"\n📋 Sample of final modeling dataset:")
#modeling_dataset.select("ICUSTAY_ID", "ICU_LOS_DAYS", "AGE_AT_ICU_ADMISSION", 
#                        "HEART_RATE_AVG", "CREATININE_FIRST", "HAS_SEPSIS").show(5)

# Basic statistics of target variable
#print(f"\n📈 Final ICU Length of Stay Statistics:")
#modeling_dataset.select("ICU_LOS_DAYS").describe().show()

print(f"\n⏰ Dataset preparation completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🚀 Ready for train/test split and model training!")
""


📋 Step 3: Selecting final features for regression modeling...


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `ICU_LOS_DAYS` cannot be resolved. Did you mean one of the following? [`ICU_LOS`, `ICU_INTIME`, `ICU_OUTTIME`, `SPO2_MAX`, `DBP_MAX`].;
'Project [ICUSTAY_ID#304374, 'ICU_LOS_DAYS, AGE_AT_ICU_ADMISSION#305970L, GENDER_BINARY#305991, 'IS_EMERGENCY_ADMISSION, 'IS_ELECTIVE_ADMISSION, CAME_FROM_ER#306013, 'HAS_MEDICARE, 'IS_WHITE_ETHNICITY, 'ADMISSION_TO_ICU_HOURS, 'WEEKEND_ADMISSION, 'NIGHT_ADMISSION, 'FIRST_UNIT_MICU, 'FIRST_UNIT_SICU, 'FIRST_UNIT_CSRU, 'FIRST_UNIT_CCU, 'FIRST_UNIT_TSICU, CHANGED_ICU_UNIT#306138, HEART_RATE_AVG#9099, SBP_AVG#9199, DBP_AVG#9378, RESP_RATE_AVG#9562, TEMPERATURE_AVG#9751, SPO2_AVG#9945, ... 17 more fields]
+- Project [HADM_ID#304373, ICUSTAY_ID#304374, SUBJECT_ID#304372, ICU_LOS#305945, ICU_INTIME#305946, ICU_OUTTIME#305947, AGE_AT_ICU_ADMISSION#305970L, GENDER_BINARY#305991, CAME_FROM_ER#306013, HAS_INSURANCE#306036, ADMISSION_TYPE_ENCODED#306060, ETHNICITY_ENCOD

## Preparing for Machine Learning

In [ ]:
print("📊 Step 1: Creating train/test split...")

# Split the data (80% train, 20% test)
train_data, test_data = modeling_dataset.randomSplit([0.8, 0.2], seed=42)

# Cache both datasets for performance
train_data.cache()
test_data.cache()


print(f"✅ Data split completed:")
#print(f"   📈 Training set: {train_data.count():,} ICU stays ({train_data.count()/modeling_dataset.count()*100:.1f}%)")
#print(f"   📊 Test set: {test_data.count():,} ICU stays ({test_data.count()/modeling_dataset.count()*100:.1f}%)")

# Show target variable distribution in both sets
print(f"\n📈 Target variable distribution:")
print(f"Training set LOS statistics:")
train_data.select("ICU_LOS_DAYS").describe().show()

print(f"Test set LOS statistics:")
test_data.select("ICU_LOS_DAYS").describe().show()

# ============================================================================
# FEATURE VECTOR ASSEMBLY
# ============================================================================

print("\n🔧 Step 2: Assembling feature vectors...")

# Create feature vector assembler
feature_assembler = VectorAssembler(
    inputCols=feature_columns,
    outputCol="features_raw"
)

# Apply feature assembler to training data
train_assembled = feature_assembler.transform(train_data)
test_assembled = feature_assembler.transform(test_data)


train_data.unpersist()
test_data.unpersist()


print(f"✅ Feature vectors assembled:")
print(f"   📊 Feature vector size: {len(feature_columns)} dimensions")

# ============================================================================
# FEATURE SCALING
# ============================================================================

print("\n⚖️ Step 3: Scaling features...")

# Create StandardScaler to normalize features
scaler = StandardScaler(
    inputCol="features_raw",
    outputCol="features",
    withStd=True,
    withMean=True
)

# Fit scaler on training data
scaler_model = scaler.fit(train_assembled)
train_scaled = scaler_model.transform(train_assembled)
test_scaled = scaler_model.transform(test_assembled)

# Cache the final processed datasets
train_scaled.cache()
test_scaled.cache()


print(f"✅ Feature scaling completed:")
print(f"   📊 Features standardized (mean=0, std=1)")
print(f"   🔧 Scaler fitted on training data only")


📊 Step 1: Creating train/test split...
✅ Data split completed:

📈 Target variable distribution:
Training set LOS statistics:


+-------+------------------+
|summary|      ICU_LOS_DAYS|
+-------+------------------+
|  count|                12|
|   mean|2.6414416666666667|
| stddev|2.2480063213508394|
|    min|             0.758|
|    max|            8.9163|
+-------+------------------+

Test set LOS statistics:


+-------+-------------------+
|summary|       ICU_LOS_DAYS|
+-------+-------------------+
|  count|                  4|
|   mean|           1.223325|
| stddev|0.42495704390757744|
|    min|              0.848|
|    max|             1.8064|
+-------+-------------------+


🔧 Step 2: Assembling feature vectors...
✅ Feature vectors assembled:
   📊 Feature vector size: 39 dimensions

⚖️ Step 3: Scaling features...


✅ Feature scaling completed:
   📊 Features standardized (mean=0, std=1)
   🔧 Scaler fitted on training data only


## Final Dataset Preparation

In [ ]:

print("\n📋 Step 4: Preparing final ML datasets...")

# Select columns needed for modeling
ml_columns = ["ICUSTAY_ID", "ICU_LOS_DAYS", "features"]

train_final = train_scaled.select(ml_columns).withColumnRenamed("ICU_LOS_DAYS", "label")
test_final = test_scaled.select(ml_columns).withColumnRenamed("ICU_LOS_DAYS", "label")

#train_final = train_assembled.select(ml_columns).withColumnRenamed("ICU_LOS_DAYS", "label")
#test_final = test_assembled.select(ml_columns).withColumnRenamed("ICU_LOS_DAYS", "label")


# train_final = train_assembled.select("ICUSTAY_ID", "ICU_LOS_DAYS", "features_raw") \
#     .withColumnRenamed("ICU_LOS_DAYS", "label") \
#     .withColumnRenamed("features_raw", "features")

# test_final = test_assembled.select("ICUSTAY_ID", "ICU_LOS_DAYS", "features_raw") \
#     .withColumnRenamed("ICU_LOS_DAYS", "label") \
#     .withColumnRenamed("features_raw", "features")




print("\n📋 Caching...")

# Cache final datasets
train_final.cache()
test_final.cache()

print(f"✅ Final ML datasets prepared:")
print(f"   🎯 Target variable: 'label' (ICU_LOS_DAYS)")
print(f"   📊 Features: 'features' (scaled vector)")
print(f"   🔑 Identifier: 'ICUSTAY_ID'")

# Show sample of final datasets
print(f"\n📋 Sample of training data structure:")
train_final.select("ICUSTAY_ID", "label").show(3)

print(f"\n📋 Feature vector example (first 10 features):")
# Show first few elements of feature vector for one sample
sample_features = train_final.select("features").take(1)[0]["features"]
print(f"   📊 Feature vector sample: {sample_features.toArray()[:10]}...")
print(f"   📏 Total feature dimensions: {len(sample_features.toArray())}")

# ============================================================================
# DATA QUALITY CHECKS
# ============================================================================

print(f"\n🔍 Step 5: Final data quality checks...")

# Check for any remaining nulls
train_nulls = train_final.filter(col("label").isNull() | col("features").isNull()).count()
test_nulls = test_final.filter(col("label").isNull() | col("features").isNull()).count()

print(f"   🔍 Null values in training set: {train_nulls}")
print(f"   🔍 Null values in test set: {test_nulls}")

# Show target variable ranges
# train_stats = train_final.agg(
#     min("label").alias("min_los"),
#     max("label").alias("max_los"), 
#     avg("label").alias("mean_los"),
#     stddev("label").alias("std_los")
# ).collect()[0]

# print(f"\n📊 Final training set target statistics:")
# print(f"   📉 Min LOS: {train_stats['min_los']:.2f} days")
# print(f"   📈 Max LOS: {train_stats['max_los']:.2f} days") 
# print(f"   📊 Mean LOS: {train_stats['mean_los']:.2f} days")
# print(f"   📏 Std LOS: {train_stats['std_los']:.2f} days")

print(f"\n✅ Data preprocessing completed successfully!")
print(f"🚀 Ready for model training with {len(feature_columns)} features")


print(f"⏰ Preprocessing completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


📋 Step 4: Preparing final ML datasets...

📋 Caching...
✅ Final ML datasets prepared:
   🎯 Target variable: 'label' (ICU_LOS_DAYS)
   📊 Features: 'features' (scaled vector)
   🔑 Identifier: 'ICUSTAY_ID'

📋 Sample of training data structure:


+----------+------+
|ICUSTAY_ID| label|
+----------+------+
|    290009|2.8701|
|    255819| 0.758|
|    248205|  4.05|
+----------+------+
only showing top 3 rows


📋 Feature vector example (first 10 features):
   📊 Feature vector sample: [ 0.37509689  0.6770032   0.5527708  -0.5527708  -0.42817442  0.95742711
 -3.17542648 -0.58005311 -0.42817442  0.        ]...
   📏 Total feature dimensions: 39

🔍 Step 5: Final data quality checks...


   🔍 Null values in training set: 0
   🔍 Null values in test set: 0

✅ Data preprocessing completed successfully!
🚀 Ready for model training with 39 features
⏰ Preprocessing completed at: 2025-06-02 12:07:54


## Training Multiple Models

In [ ]:
print("📊 Step 1: Setting up evaluation metrics...")

# Create regression evaluators
rmse_evaluator = RegressionEvaluator(
    labelCol="label", 
    predictionCol="prediction", 
    metricName="rmse"
)

mae_evaluator = RegressionEvaluator(
    labelCol="label",
    predictionCol="prediction", 
    metricName="mae"
)

r2_evaluator = RegressionEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="r2"
)

print("✅ Evaluation metrics configured: RMSE, MAE, R²")

📊 Step 1: Setting up evaluation metrics...
✅ Evaluation metrics configured: RMSE, MAE, R²


### Linear Regression

In [ ]:
print("\n📈 Step 2: Training Linear Regression model...")
print(f"🕐 Started at: {datetime.now().strftime('%H:%M:%S')}")
start_time = time.time()

# Create Linear Regression model
lr = LinearRegression(
    featuresCol="features",
    labelCol="label",
    maxIter=200,                    # Increased for better convergence
    regParam=0.001,                 # Lower regularization for healthcare data
    elasticNetParam=0.1,            # Slight L1 penalty for feature selection
    tol=1e-8,                       # Tighter tolerance for precision
    standardization=False,          # We're doing manual scaling
    fitIntercept=True,
    aggregationDepth=3,             # Better for distributed training
    loss="squaredError",
    solver="normal"                 # Best for small-medium datasets
)


# Train the model
print("   🔄 Training Linear Regression...")
lr_model = lr.fit(train_final)

print("   🔄 Linear Regression - Making predictions (test data)...")
lr_predictions = lr_model.transform(test_final)

print("   🔄 Linear Regression - Evaluation...")
lr_rmse = rmse_evaluator.evaluate(lr_predictions)
lr_mae = mae_evaluator.evaluate(lr_predictions)
lr_r2 = r2_evaluator.evaluate(lr_predictions)

print(f"✅ Linear Regression Results:")
print(f"   📉 RMSE: {lr_rmse:.3f} days")
print(f"   📊 MAE: {lr_mae:.3f} days")
print(f"   📈 R²: {lr_r2:.3f}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"🕐 Completed at: {datetime.now().strftime('%H:%M:%S')}")
print(f"⏱️ Total elapsed time: {elapsed_time:.2f} seconds")


# Linear Regression Predictions

print("\n📈 Linear Regression Predictions (Sample 20):")
lr_display = lr_predictions.select(
    "ICUSTAY_ID",
    col("label").alias("Actual_LOS"),
    round(col("prediction"), 3).alias("Predicted_LOS"),
    round(abs(col("label") - col("prediction")), 3).alias("Absolute_Error"),
    round(((abs(col("label") - col("prediction")) / col("label")) * 100), 2).alias("Percent_Error")
).orderBy("ICUSTAY_ID")

lr_display.show(20, truncate=False)


📈 Step 2: Training Linear Regression model...
🕐 Started at: 12:07:54
   🔄 Training Linear Regression...


25/06/02 12:07:57 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


   🔄 Linear Regression - Making predictions (test data)...
   🔄 Linear Regression - Evaluation...


✅ Linear Regression Results:
   📉 RMSE: 2.272 days
   📊 MAE: 1.931 days
   📈 R²: -37.112
🕐 Completed at: 12:08:31
⏱️ Total elapsed time: 37.31 seconds

📈 Linear Regression Predictions (Sample 20):


+----------+----------+-------------+--------------+-------------+
|ICUSTAY_ID|Actual_LOS|Predicted_LOS|Absolute_Error|Percent_Error|
+----------+----------+-------------+--------------+-------------+
|231977    |0.9792    |3.914        |2.934         |299.66       |
|252713    |0.848     |3.875        |3.027         |356.99       |
|259725    |1.8064    |1.737        |0.069         |3.84         |
|298190    |1.2597    |2.953        |1.694         |134.44       |
+----------+----------+-------------+--------------+-------------+



### Random Forest

In [ ]:

print("\n🌲 Step 3: Training Random Forest model...")
print(f"🕐 Started at: {datetime.now().strftime('%H:%M:%S')}")
start_time = time.time()

# Create Random Forest model
rf = RandomForestRegressor(
    featuresCol="features",
    labelCol="label",
    numTrees=300,                   # More trees = better accuracy (if enough cores/memory)
    maxDepth=12,                    # Deeper trees capture more complexity
    minInstancesPerNode=2,          # Allows more granular splits
    subsamplingRate=0.9,            # Slightly higher sample rate for stability
    featureSubsetStrategy="sqrt",   # Good default for regression
    maxBins=64,                     # More bins = better numeric split precision
    impurity="variance",            # Required for regression
    maxMemoryInMB=512,              # Give more memory per node for splits
    cacheNodeIds=True,              # Improves tree building performance
    checkpointInterval=5,           # Frequent checkpoints = safer on big jobs
    seed=42                         # Reproducibility
)

print("   🔄 Training Random Forest...")
rf_model = rf.fit(train_final)

print("   🔄 Random Forest - Making predictions (test data)...")
rf_predictions = rf_model.transform(test_final)

print("   🔄 Random Forest - Evaluation...")
rf_rmse = rmse_evaluator.evaluate(rf_predictions)
rf_mae = mae_evaluator.evaluate(rf_predictions)
rf_r2 = r2_evaluator.evaluate(rf_predictions)

print(f"✅ Random Forest Results:")
print(f"   📉 RMSE: {rf_rmse:.3f} days")
print(f"   📊 MAE: {rf_mae:.3f} days")
print(f"   📈 R²: {rf_r2:.3f}")


end_time = time.time()
elapsed_time = end_time - start_time
print(f"🕐 Completed at: {datetime.now().strftime('%H:%M:%S')}")
print(f"⏱️ Total elapsed time: {elapsed_time:.2f} seconds")


# Random Forest Predictions
print("\n🌲 Random Forest Predictions (Sample 20):")
rf_display = rf_predictions.select(
    "ICUSTAY_ID",
    col("label").alias("Actual_LOS"),
    round(col("prediction"), 3).alias("Predicted_LOS"),
    round(abs(col("label") - col("prediction")), 3).alias("Absolute_Error"),
    round(((abs(col("label") - col("prediction")) / col("label")) * 100), 2).alias("Percent_Error")
).orderBy("ICUSTAY_ID")

rf_display.show(20, truncate=False)



🌲 Step 3: Training Random Forest model...
🕐 Started at: 12:08:39
   🔄 Training Random Forest...


25/06/02 12:08:45 WARN DecisionTreeMetadata: DecisionTree reducing maxBins from 64 to 12 (= number of training instances)


   🔄 Random Forest - Making predictions (test data)...


   🔄 Random Forest - Evaluation...


✅ Random Forest Results:
   📉 RMSE: 1.618 days
   📊 MAE: 1.551 days
   📈 R²: -18.339
🕐 Completed at: 12:10:09
⏱️ Total elapsed time: 90.14 seconds

🌲 Random Forest Predictions (Sample 20):


+----------+----------+-------------+--------------+-------------+
|ICUSTAY_ID|Actual_LOS|Predicted_LOS|Absolute_Error|Percent_Error|
+----------+----------+-------------+--------------+-------------+
|231977    |0.9792    |2.538        |1.559         |159.22       |
|252713    |0.848     |2.919        |2.071         |244.24       |
|259725    |1.8064    |2.62         |0.814         |45.05        |
|298190    |1.2597    |3.019        |1.759         |139.64       |
+----------+----------+-------------+--------------+-------------+



## Model Comparison

In [ ]:
print("\n🏆 Step 5: Model Performance Comparison...")

# Create comparison summary
results_data = [
    ("Linear Regression", lr_rmse, lr_mae, lr_r2),
    ("Random Forest", rf_rmse, rf_mae, rf_r2)
]

results_df = spark.createDataFrame(results_data, ["Model", "RMSE", "MAE", "R2"])

print("📊 Model Performance Summary:")
results_df.show(truncate=False)

# Find best model
import operator
import builtins
best_rmse_model = builtins.min(results_data, key=operator.itemgetter(1))
best_r2_model = builtins.max(results_data, key=operator.itemgetter(3))

print(f"\n🥇 Best Models:")
print(f"   🎯 Lowest RMSE: {best_rmse_model[0]} ({best_rmse_model[1]:.3f} days)")
print(f"   📈 Highest R²: {best_r2_model[0]} ({best_r2_model[3]:.3f})")


🏆 Step 5: Model Performance Comparison...
📊 Model Performance Summary:


+-----------------+------------------+------------------+-------------------+
|Model            |RMSE              |MAE               |R2                 |
+-----------------+------------------+------------------+-------------------+
|Linear Regression|2.2719759149846968|1.931127670467299 |-37.1115066790081  |
|Random Forest    |1.618407382026571 |1.5507799916666642|-18.338570738254734|
+-----------------+------------------+------------------+-------------------+


🥇 Best Models:
   🎯 Lowest RMSE: Random Forest (1.618 days)
   📈 Highest R²: Random Forest (-18.339)


## Display Predictions

In [ ]:

# Linear Regression Predictions

print("\n📈 Linear Regression Predictions (Sample 20):")
lr_display = lr_predictions.select(
    "ICUSTAY_ID",
    col("label").alias("Actual_LOS"),
    round(col("prediction"), 3).alias("Predicted_LOS"),
    round(abs(col("label") - col("prediction")), 3).alias("Absolute_Error"),
    round(((abs(col("label") - col("prediction")) / col("label")) * 100), 2).alias("Percent_Error")
).orderBy("ICUSTAY_ID")

lr_display.show(20, truncate=False)



# Random Forest Predictions
print("\n🌲 Random Forest Predictions (Sample 20):")
rf_display = rf_predictions.select(
    "ICUSTAY_ID",
    col("label").alias("Actual_LOS"),
    round(col("prediction"), 3).alias("Predicted_LOS"),
    round(abs(col("label") - col("prediction")), 3).alias("Absolute_Error"),
    round(((abs(col("label") - col("prediction")) / col("label")) * 100), 2).alias("Percent_Error")
).orderBy("ICUSTAY_ID")

rf_display.show(20, truncate=False)


📈 Linear Regression Predictions (Sample 20):


+----------+----------+-------------+--------------+-------------+
|ICUSTAY_ID|Actual_LOS|Predicted_LOS|Absolute_Error|Percent_Error|
+----------+----------+-------------+--------------+-------------+
|231977    |0.9792    |3.914        |2.934         |299.66       |
|252713    |0.848     |3.875        |3.027         |356.99       |
|259725    |1.8064    |1.737        |0.069         |3.84         |
|298190    |1.2597    |2.953        |1.694         |134.44       |
+----------+----------+-------------+--------------+-------------+


🌲 Random Forest Predictions (Sample 20):


+----------+----------+-------------+--------------+-------------+
|ICUSTAY_ID|Actual_LOS|Predicted_LOS|Absolute_Error|Percent_Error|
+----------+----------+-------------+--------------+-------------+
|231977    |0.9792    |2.538        |1.559         |159.22       |
|252713    |0.848     |2.919        |2.071         |244.24       |
|259725    |1.8064    |2.62         |0.814         |45.05        |
|298190    |1.2597    |3.019        |1.759         |139.64       |
+----------+----------+-------------+--------------+-------------+

